In [1]:
import pandas as pd

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASS = os.getenv('NEO4J_PASS')
NEO4J_USERNAME, NEO4J_PASS

('neo4j', 'password')

In [16]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "bolt://localhost:7687"
AUTH = (NEO4J_USERNAME, NEO4J_PASS)

driver = GraphDatabase.driver(URI, auth=AUTH)

def get_session(database_name="neo4j"):
    # Create a session and specify the database
    return driver.session(database=database_name)

In [17]:
with get_session() as session:
    result = session.run("RETURN 'Connection successful!'")
    print(result.single()[0])  # Should print "Connection successful!"

Connection successful!


In [4]:
df = pd.read_csv("fraud_data_10k.csv")
df.head()

,user,uniqueidentifier1,uniqueidentifier2,uniqueidentifier3,Fraud
0,User_1,4ccc2dc9-5425-4825-9141-4c27b3d5fadf,V9481,156537,0
1,User_2,1c4f19ad-e7bf-4942-9e6f-cd35f0f17e50,N2400,118492,1
2,User_3,d1f0223c-38d0-4c4a-8aa8-ff604b694cb9,P1073,133863,0
3,User_4,fcb8c3fa-9874-4cbf-b00c-cd7c4f935dc4,Y8631,132353,0
4,User_5,fdbe86d5-0ff7-4b70-a324-6dba7d189724,I9614,148025,1


In [5]:
df[df['uniqueidentifier1']=="4ccc2dc9-5425-4825-9141-4c27b3d5fadf"]

,user,uniqueidentifier1,uniqueidentifier2,uniqueidentifier3,Fraud
0,User_1,4ccc2dc9-5425-4825-9141-4c27b3d5fadf,V9481,156537,0


In [7]:
def batch_create_nodes(df, label, column, batch_size=10000):
    with get_session() as session:
        for i in range(0, len(df), batch_size):
            batch = df.iloc[i:i+batch_size]
            records = batch[[column]].drop_duplicates().rename(columns={column: "id"}).dropna()
            
            def create_nodes(tx, rows):
                tx.run(f"""
                    UNWIND $rows AS row
                    MERGE (n:{label} {{id: row.id}})
                """, parameters={"rows": rows})

            session.execute_write(create_nodes, records.to_dict("records"))


batch_create_nodes(df, "User", "user")
batch_create_nodes(df, "UID1", "uniqueidentifier1")
batch_create_nodes(df, "UID2", "uniqueidentifier2")
batch_create_nodes(df, "UID3", "uniqueidentifier3")

In [8]:
with get_session() as session:
    graph_response = session.run("""MATCH (n:User) RETURN count(n);""")
    print([record.data() for record in graph_response])

[{'count(n)': 10000}]


In [9]:
# with get_session() as session:
#     graph_response = session.run("""MATCH (n) DETACH DELETE n""")
#     print([record.data() for record in graph_response])

In [10]:
def batch_create_relationships(df, batch_size=10000):
    with get_session() as session:
        for i in range(0, len(df), batch_size):
            batch = df.iloc[i:i+batch_size].dropna(subset=["user", "uniqueidentifier1", "uniqueidentifier2", "uniqueidentifier3"])

            rows = batch[["user", "uniqueidentifier1", "uniqueidentifier2", "uniqueidentifier3"]].to_dict("records")

            def create_relationships(tx, rows):
                tx.run("""
                    UNWIND $rows AS row
                    MATCH (u:User {id: row.user})
                    MATCH (uid1:UID1 {id: row.uniqueidentifier1})
                    MATCH (uid2:UID2 {id: row.uniqueidentifier2})
                    MATCH (uid3:UID3 {id: row.uniqueidentifier3})
                    MERGE (u)-[:HAS_UID1]->(uid1)
                    MERGE (u)-[:HAS_UID2]->(uid2)
                    MERGE (u)-[:HAS_UID3]->(uid3)
                """, parameters={"rows": rows})

            session.execute_write(create_relationships, rows)

batch_create_relationships(df)

In [11]:
with get_session() as session:
    graph_response = session.run("""MATCH (u:User)-[r]->(n) RETURN u.id, type(r), n.id LIMIT 20;""")
    print([record.data() for record in graph_response])

[{'u.id': 'User_1', 'type(r)': 'HAS_UID1', 'n.id': '4ccc2dc9-5425-4825-9141-4c27b3d5fadf'}, {'u.id': 'User_1', 'type(r)': 'HAS_UID2', 'n.id': 'V9481'}, {'u.id': 'User_1', 'type(r)': 'HAS_UID3', 'n.id': 156537}, {'u.id': 'User_2', 'type(r)': 'HAS_UID1', 'n.id': '1c4f19ad-e7bf-4942-9e6f-cd35f0f17e50'}, {'u.id': 'User_2', 'type(r)': 'HAS_UID2', 'n.id': 'N2400'}, {'u.id': 'User_2', 'type(r)': 'HAS_UID3', 'n.id': 118492}, {'u.id': 'User_3', 'type(r)': 'HAS_UID1', 'n.id': 'd1f0223c-38d0-4c4a-8aa8-ff604b694cb9'}, {'u.id': 'User_3', 'type(r)': 'HAS_UID2', 'n.id': 'P1073'}, {'u.id': 'User_3', 'type(r)': 'HAS_UID3', 'n.id': 133863}, {'u.id': 'User_4', 'type(r)': 'HAS_UID1', 'n.id': 'fcb8c3fa-9874-4cbf-b00c-cd7c4f935dc4'}, {'u.id': 'User_4', 'type(r)': 'HAS_UID2', 'n.id': 'Y8631'}, {'u.id': 'User_4', 'type(r)': 'HAS_UID3', 'n.id': 132353}, {'u.id': 'User_5', 'type(r)': 'HAS_UID1', 'n.id': 'fdbe86d5-0ff7-4b70-a324-6dba7d189724'}, {'u.id': 'User_5', 'type(r)': 'HAS_UID2', 'n.id': 'I9614'}, {'u.id':

In [12]:
def label_fraudulent_users(df, batch_size=10000):
    with get_session() as session:
        fraud_df = df[df["Fraud"] == 1]

        for i in range(0, len(fraud_df), batch_size):
            batch = fraud_df.iloc[i:i+batch_size]
            users = batch["user"].drop_duplicates().to_frame(name="id").dropna()

            def set_fraud_label(tx, rows):
                tx.run("""
                    UNWIND $rows AS row
                    MATCH (u:User {id: row.id})
                    SET u:Fraud
                """, parameters={"rows": rows})

            session.execute_write(set_fraud_label, users.to_dict("records"))

label_fraudulent_users(df)

In [13]:
with get_session() as session:
    graph_response = session.run("""MATCH (u:User:Fraud) RETURN u.id LIMIT 10;""")
    print([record.data() for record in graph_response])

[{'u.id': 'User_2'}, {'u.id': 'User_5'}, {'u.id': 'User_16'}, {'u.id': 'User_23'}, {'u.id': 'User_40'}, {'u.id': 'User_44'}, {'u.id': 'User_54'}, {'u.id': 'User_68'}, {'u.id': 'User_73'}, {'u.id': 'User_97'}]


In [57]:
user_id = "User_23"
edges = ['HAS_UID2','HAS_UID1']
level = 2


query = f"""
    MATCH path=(:User {{id: '{user_id}'}})-[:{"|".join(edges)}*..{level}]-(connected:User)
    WHERE ALL(r IN relationships(path) WHERE type(r) in {edges})
    AND length(path) > 0
    AND connected.id <> "User_23"
    RETURN connected, length(path) AS connectionDistance
    ORDER BY connectionDistance
    """
print(query)

with get_session() as session:
    result = session.run(query, parameters={"user_id": user_id, "edges":edges, "level":level})
    result = result.data()


    MATCH path=(:User {id: 'User_23'})-[:HAS_UID2|HAS_UID1*..2]-(connected:User)
    WHERE ALL(r IN relationships(path) WHERE type(r) in ['HAS_UID2', 'HAS_UID1'])
    AND length(path) > 0
    AND connected.id <> "User_23"
    RETURN connected, length(path) AS connectionDistance
    ORDER BY connectionDistance
    


In [61]:
# FINDING CONNECTIONS
def get_connected_users_by_level(user_id, edges, level):
    if len(edges)==1:
        edges = edges[0]
        query = f"""
            MATCH path=(:User {{id: '{user_id}'}})-[:{edges}*..{level}]-(connected:User)
            WHERE ALL(r IN relationships(path) WHERE type(r) in ['{edges}'])
            AND length(path) > 0
            AND connected.id <> "User_23"
            RETURN connected, length(path) AS connectionDistance
            ORDER BY connectionDistance
            """
    else:
        query = f"""
            MATCH path=(:User {{id: '{user_id}'}})-[:{"|".join(edges)}*..{level}]-(connected:User)
            WHERE ALL(r IN relationships(path) WHERE type(r) in {edges})
            AND length(path) > 0
            AND connected.id <> "User_23"
            RETURN connected, length(path) AS connectionDistance
            ORDER BY connectionDistance
            """
    with get_session() as session:
        result = session.run(query, parameters={"user_id": user_id, "edges":edges, "level":level})
        # print(query)
        return result.data()
    
result = get_connected_users_by_level("User_23",['HAS_UID1'],2)
result

[{'connected': {'componentId': 3, 'id': 'User_19'}, 'connectionDistance': 2},
 {'connected': {'componentId': 3, 'id': 'User_80'}, 'connectionDistance': 2},
 {'connected': {'componentId': 3, 'id': 'User_94'}, 'connectionDistance': 2},
 {'connected': {'componentId': 3, 'id': 'User_503'}, 'connectionDistance': 2},
 {'connected': {'componentId': 3, 'id': 'User_579'}, 'connectionDistance': 2},
 {'connected': {'componentId': 3, 'id': 'User_1127'}, 'connectionDistance': 2},
 {'connected': {'componentId': 3, 'id': 'User_1207'}, 'connectionDistance': 2},
 {'connected': {'componentId': 3, 'id': 'User_1783'}, 'connectionDistance': 2},
 {'connected': {'componentId': 3, 'id': 'User_1870'}, 'connectionDistance': 2},
 {'connected': {'componentId': 3, 'id': 'User_2371'}, 'connectionDistance': 2},
 {'connected': {'componentId': 3, 'id': 'User_2391'}, 'connectionDistance': 2},
 {'connected': {'componentId': 3, 'id': 'User_2416'}, 'connectionDistance': 2},
 {'connected': {'componentId': 3, 'id': 'User_33

In [ ]:
# def project_graph():
#     query = """
#     CALL gds.graph.project(
#       'fraudGraph',
#       ['User', 'UID1', 'UID2', 'UID3'],
#       {
#         HAS_UID1: {type: 'HAS_UID1', orientation: 'UNDIRECTED'},
#         HAS_UID2: {type: 'HAS_UID2', orientation: 'UNDIRECTED'},
#         HAS_UID3: {type: 'HAS_UID3', orientation: 'UNDIRECTED'}
#       }
#     )
#     YIELD graphName, nodeCount, relationshipCount
#     """
#     with get_session() as session:
#         result = session.run(query)
#         return result.single()

# project_graph()

# def run_wcc():
#     query = """
#     CALL gds.wcc.write(
#       'fraudGraph',
#       {
#         writeProperty: 'componentId'
#       }
#     )
#     YIELD componentCount, nodePropertiesWritten
#     """
#     with get_session() as session:
#         result = session.run(query)
#         return result.single()

# run_wcc()

# def drop_graph():
#     query = "CALL gds.graph.drop('fraudGraph') YIELD graphName"
#     with get_session() as session:
#         session.run(query)

# drop_graph()

In [ ]:
def get_connected_users_by_level(user_id, level):
    query = """
    MATCH (start:User {id: $user_id})
    CALL {
        WITH start
        MATCH path = (start)-[:HAS_UID1*1..$level]-(connected:User)
        RETURN collect(DISTINCT connected.id) AS connectedUsers
    }
    RETURN connectedUsers
    """
    with get_session() as session:
        result = session.run(query, parameters={"user_id": user_id, "level": level})
        return result.single()["connectedUsers"]

# Example usage:
level_1_users = get_connected_users_by_level("U123", 1)
print("Level 1:", level_1_users)

level_2_users = get_connected_users_by_level("U123", 2)
print("Level 2:", level_2_users)

level_5_users = get_connected_users_by_level("U123", 5)
print("Level 5:", level_5_users)


In [25]:
def get_connected_users(user_id, level, edge_types):
    # Build relationship types string — no backticks inside pipes
    rel_types_cypher = "|".join(edge_types)

    # Build full Cypher query
    query = f"""
    MATCH (start:User {{id: $user_id}})
    CALL {{
        WITH start
        MATCH path = (start)-[:{rel_types_cypher}*1..{level}]-(connected:User)
        RETURN collect(DISTINCT connected.id) AS connectedUsers
    }}
    RETURN connectedUsers
    """
    print(query)
    with get_session() as session:
        result = session.run(query, parameters={"user_id": user_id})
        return result.single()["connectedUsers"]

# Example usage:
connected_lvl1_uid1 = get_connected_users("User_23", 1, ["HAS_UID1"])
print("Level 1 UID1:", connected_lvl1_uid1)

connected_lvl3_uid1_uid2 = get_connected_users("User_23", 3, ["HAS_UID1", "HAS_UID2"])
print("Level 3 UID1 & UID2:", connected_lvl3_uid1_uid2)


# # Example usage:
# connected_lvl1_uid1 = get_connected_users("User_23", 1, ["HAS_UID1"])
# print("Level 1 UID1:", connected_lvl1_uid1)

# connected_lvl2_uid1_uid2 = get_connected_users("User_23", 2, ["HAS_UID1", "HAS_UID2"])
# print("Level 2 UID1 & UID2:", connected_lvl2_uid1_uid2)

# connected_lvl2_uid1_uid2 = get_connected_users("User_23", 4, ["HAS_UID1", "HAS_UID2"])
# print("Level 2 UID1 & UID2:", connected_lvl2_uid1_uid2)

# connected_all_5 = get_connected_users("User_23", 5, ["HAS_UID1", "HAS_UID2", "HAS_UID3"])
# print("Level 5 UID1, UID2, UID3:", connected_all_5)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (start) { ... }} {position: line: 3, column: 5, offset: 43} for query: '\n    MATCH (start:User {id: $user_id})\n    CALL {\n        WITH start\n        MATCH path = (start)-[:HAS_UID1*1..1]-(connected:User)\n        RETURN collect(DISTINCT connected.id) AS connectedUsers\n    }\n    RETURN connectedUsers\n    '


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (start) { ... }} {position: line: 3, column: 5, offset: 43} for query: '\n    MATCH (start:User {id: $user_id})\n    CALL {\n        WITH start\n        MATCH path = (start)-[:HAS_UID1|HAS_UID2*1..3]-(connected:User)\n        RETURN collect(DISTINCT connected.id) AS connectedUsers\n    }\n    RETURN connectedUsers\n    '



    MATCH (start:User {id: $user_id})
    CALL {
        WITH start
        MATCH path = (start)-[:HAS_UID1*1..1]-(connected:User)
        RETURN collect(DISTINCT connected.id) AS connectedUsers
    }
    RETURN connectedUsers
    
Level 1 UID1: []

    MATCH (start:User {id: $user_id})
    CALL {
        WITH start
        MATCH path = (start)-[:HAS_UID1|HAS_UID2*1..3]-(connected:User)
        RETURN collect(DISTINCT connected.id) AS connectedUsers
    }
    RETURN connectedUsers
    
Level 3 UID1 & UID2: ['User_174', 'User_1117', 'User_1394', 'User_1467', 'User_1912', 'User_1968', 'User_2167', 'User_2243', 'User_2385', 'User_2840', 'User_3462', 'User_4211', 'User_4243', 'User_4305', 'User_4406', 'User_4508', 'User_4697', 'User_5113', 'User_5244', 'User_5796', 'User_6339', 'User_7103', 'User_7776', 'User_7864', 'User_7958', 'User_8321', 'User_8804', 'User_8849', 'User_8976', 'User_9012', 'User_9025', 'User_9292', 'User_9541', 'User_9656', 'User_19', 'User_80', 'User_94', 'User_503', 

In [19]:
def get_connected_users(user_id):
    query = """
    MATCH (u:User {id: $user_id})
    WITH u.componentId AS compId
    MATCH (other:User)
    WHERE other.componentId = compId
    RETURN other.id AS ConnectedUser
    """
    with get_session() as session:
        result = session.run(query, parameters={"user_id": user_id})
        return [record["ConnectedUser"] for record in result]

fraud_ring = get_connected_users("User_23")
print(fraud_ring)


['User_4', 'User_5', 'User_7', 'User_9', 'User_10', 'User_13', 'User_16', 'User_17', 'User_18', 'User_19', 'User_20', 'User_21', 'User_23', 'User_25', 'User_26', 'User_28', 'User_32', 'User_34', 'User_36', 'User_37', 'User_38', 'User_41', 'User_42', 'User_46', 'User_50', 'User_54', 'User_57', 'User_60', 'User_64', 'User_65', 'User_66', 'User_67', 'User_68', 'User_69', 'User_71', 'User_75', 'User_77', 'User_80', 'User_84', 'User_85', 'User_90', 'User_91', 'User_94', 'User_95', 'User_96', 'User_98', 'User_99', 'User_101', 'User_102', 'User_106', 'User_108', 'User_112', 'User_113', 'User_114', 'User_115', 'User_116', 'User_118', 'User_121', 'User_122', 'User_123', 'User_126', 'User_127', 'User_128', 'User_129', 'User_130', 'User_131', 'User_132', 'User_137', 'User_139', 'User_140', 'User_142', 'User_150', 'User_152', 'User_154', 'User_156', 'User_157', 'User_158', 'User_159', 'User_160', 'User_162', 'User_163', 'User_165', 'User_166', 'User_167', 'User_170', 'User_173', 'User_174', 'User_

In [21]:
def get_connected_users_via_uid1_recursive(user_id):
    query = """
    MATCH path = (u:User {id: $user_id})-[:CONNECTED_TO*1..]->(uid:UID1)<-[:CONNECTED_TO*1..]-(other:User)
    RETURN DISTINCT other.id AS ConnectedUser
    """
    with get_session() as session:
        result = session.run(query, parameters={"user_id": user_id})
        return [record["ConnectedUser"] for record in result]

# Example usage
connected_users = get_connected_users_via_uid1_recursive("User_23")
print(connected_users)


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: CONNECTED_TO)} {position: line: 2, column: 44, offset: 44} for query: '\n    MATCH path = (u:User {id: $user_id})-[:CONNECTED_TO*1..]->(uid:UID1)<-[:CONNECTED_TO*1..]-(other:User)\n    RETURN DISTINCT other.id AS ConnectedUser\n    '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the dat

[]
